<a href="https://colab.research.google.com/github/gmfv/YIO/blob/GioM/DatosArreglar_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importamos las librerías numpy, pandas y matplotlib 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sb

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/gmfv/YIO/main/DatosHogares%20-%20Poblaci%C3%B3n%2C%20hogares%20por%20a%C3%B1o%20de%20la%20encuesta%2C%20seg%C3%BAn%20%20departamento%20y%20acceso%20a%20servicios%20b%C3%A1sicos%20(1).csv")

3/ Datos no disponibles

**Arreglamos el Dataset**
1. Crear columna Departamento
2. Rellenar los 3/ por algo

In [ ]:
copy_df=df.copy()
copy_df = copy_df.rename(columns={'Departamento y acceso a servicios basicos':'Acceso'})
copy_df=copy_df.loc[(copy_df.Acceso != "Porcentaje")]
copy_df=copy_df.loc[(copy_df.Acceso != "Valor absoluto")]

In [ ]:
copy_df = copy_df.replace("3/", 0)

In [ ]:
Departamentos=[]
for i in copy_df['Acceso'].apply(lambda x: x.split()):
  if len(i) == 1:
    Departamentos.append(i[0])
  else:
    Departamentos.append(np.nan)
copy_df['Departamento']=Departamentos
copy_df['Departamento']=copy_df['Departamento'].fillna(method='pad')
copy_df=copy_df.dropna()
copy_df=copy_df.set_index('Departamento')#[copy_df.index, 'Departamento'])
copy_df=copy_df.reset_index()

,Departamento,Acceso,1997/98,1999,2000/01,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Asuncion,Poblacion con acceso a saneamiento mejorado (%),"93,4","93,9","95,3",94,"91,7","93,1","94,4","90,8","94,9",90,"95,9","92,8","90,8","96,1","95,1","95,4","92,1","94,6","92,4","95,6"
1,Asuncion,Poblacion con acceso a agua mejorada (%),"96,8","96,9","96,2","98,8","97,6","98,9","97,4","97,6","99,4","98,2","98,8","99,1","96,5","99,7","99,6",98,"98,8","99,5","99,7","98,1"
2,Asuncion,Hogares con acceso a electricidad (%),99,"99,7","98,7","99,6","99,6","98,3","99,5","99,3","99,4","99,5","99,4","99,8","99,9",100,100,100,100,"99,9","99,9","99,8"
3,Asuncion,Poblacion con acceso a saneamiento mejorado,490.042,494.570,503.869,498.432,486.864,494.446,501.694,482.836,504.642,478.277,509.786,492.850,481.381,508.650,502.890,502.951,484.721,497.111,484.380,500.031
4,Asuncion,Poblacion con acceso a agua mejorada,507.593,510.121,508.473,524.288,518.197,525.202,517.844,519.097,528.514,521.838,525.214,525.948,511.699,527.829,526.267,516.673,520.121,522.468,522.614,513.482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Boqueron,Poblacion con acceso a agua mejorada (%),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"62,9","82,3","78,3",0
104,Boqueron,Hogares con acceso a electricidad (%),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"92,4","89,6","94,2",0
105,Boqueron,Poblacion con acceso a saneamiento mejorado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.703,4.616,4.569,0
106,Boqueron,Poblacion con acceso a agua mejorada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.438,13.900,13.474,0


In [ ]:
copy_df.reset_index().iloc[9][3::].values
copy_df=copy_df.apply(lambda x: x.astype(str).str.replace('.',''))
copy_df=copy_df.apply(lambda x: x.astype(str).str.replace(',','.'))


In [119]:
copy_df=copy_df.apply(pd.to_numeric, errors='ignore')
copy_df = copy_df.replace(0, np.NaN)
copy_df['Promedio']=copy_df.mean(axis=1, skipna=True, numeric_only=True)
for i in range (len(copy_df.Acceso)):
  copy_df.iloc[i] = copy_df.iloc[i].fillna(copy_df.iloc[i,-1]) 


,Departamento,Acceso,1997/98,1999,2000/01,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Promedio
0,Asuncion,Poblacion con acceso a saneamiento mejorado (%),93.400000,93.900000,95.300000,94.000000,91.700000,93.100000,94.400000,90.800000,94.900000,90.000000,95.900000,92.800000,90.800000,96.100000,95.100000,95.400000,92.1,94.6,92.4,95.600000,93.615000
1,Asuncion,Poblacion con acceso a agua mejorada (%),96.800000,96.900000,96.200000,98.800000,97.600000,98.900000,97.400000,97.600000,99.400000,98.200000,98.800000,99.100000,96.500000,99.700000,99.600000,98.000000,98.8,99.5,99.7,98.100000,98.280000
2,Asuncion,Hogares con acceso a electricidad (%),99.000000,99.700000,98.700000,99.600000,99.600000,98.300000,99.500000,99.300000,99.400000,99.500000,99.400000,99.800000,99.900000,100.000000,100.000000,100.000000,100.0,99.9,99.9,99.800000,99.565000
3,Asuncion,Poblacion con acceso a saneamiento mejorado,490042.000000,494570.000000,503869.000000,498432.000000,486864.000000,494446.000000,501694.000000,482836.000000,504642.000000,478277.000000,509786.000000,492850.000000,481381.000000,508650.000000,502890.000000,502951.000000,484721.0,497111.0,484380.0,500031.000000,495021.150000
4,Asuncion,Poblacion con acceso a agua mejorada,507593.000000,510121.000000,508473.000000,524288.000000,518197.000000,525202.000000,517844.000000,519097.000000,528514.000000,521838.000000,525214.000000,525948.000000,511699.000000,527829.000000,526267.000000,516673.000000,520121.0,522468.0,522614.0,513482.000000,519674.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Boqueron,Poblacion con acceso a agua mejorada (%),74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,62.9,82.3,78.3,74.500000,74.500000
104,Boqueron,Hogares con acceso a electricidad (%),92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.4,89.6,94.2,92.066667,92.066667
105,Boqueron,Poblacion con acceso a saneamiento mejorado,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,5703.0,4616.0,4569.0,4962.666667,4962.666667
106,Boqueron,Poblacion con acceso a agua mejorada,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,10438.0,13900.0,13474.0,12604.000000,12604.000000


In [121]:
copy_df=copy_df.drop(columns=['Promedio'])

,Departamento,Acceso,1997/98,1999,2000/01,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Asuncion,Poblacion con acceso a saneamiento mejorado (%),93.400000,93.900000,95.300000,94.000000,91.700000,93.100000,94.400000,90.800000,94.900000,90.000000,95.900000,92.800000,90.800000,96.100000,95.100000,95.400000,92.1,94.6,92.4,95.600000
1,Asuncion,Poblacion con acceso a agua mejorada (%),96.800000,96.900000,96.200000,98.800000,97.600000,98.900000,97.400000,97.600000,99.400000,98.200000,98.800000,99.100000,96.500000,99.700000,99.600000,98.000000,98.8,99.5,99.7,98.100000
2,Asuncion,Hogares con acceso a electricidad (%),99.000000,99.700000,98.700000,99.600000,99.600000,98.300000,99.500000,99.300000,99.400000,99.500000,99.400000,99.800000,99.900000,100.000000,100.000000,100.000000,100.0,99.9,99.9,99.800000
3,Asuncion,Poblacion con acceso a saneamiento mejorado,490042.000000,494570.000000,503869.000000,498432.000000,486864.000000,494446.000000,501694.000000,482836.000000,504642.000000,478277.000000,509786.000000,492850.000000,481381.000000,508650.000000,502890.000000,502951.000000,484721.0,497111.0,484380.0,500031.000000
4,Asuncion,Poblacion con acceso a agua mejorada,507593.000000,510121.000000,508473.000000,524288.000000,518197.000000,525202.000000,517844.000000,519097.000000,528514.000000,521838.000000,525214.000000,525948.000000,511699.000000,527829.000000,526267.000000,516673.000000,520121.0,522468.0,522614.0,513482.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Boqueron,Poblacion con acceso a agua mejorada (%),74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,74.500000,62.9,82.3,78.3,74.500000
104,Boqueron,Hogares con acceso a electricidad (%),92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.066667,92.4,89.6,94.2,92.066667
105,Boqueron,Poblacion con acceso a saneamiento mejorado,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,4962.666667,5703.0,4616.0,4569.0,4962.666667
106,Boqueron,Poblacion con acceso a agua mejorada,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,12604.000000,10438.0,13900.0,13474.0,12604.000000


In [ ]:
date_time = pd.to_datetime(copy_df.columns[5::], format='%Y')

plot_cols = copy_df.columns[5::]
plot_features = copy_df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)
#plot_features

ValueError: ignored

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

date_time = pd.to_datetime(copy_df.columns, format='%d.%m.%Y %H:%M:%S')

class Valores(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]



single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you 
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1)

wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1)

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

valores = Valores()
valores.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])